In [1]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from src.Utils.K_Fold import K_Fold
from src.data_loader import load_data
from src.analysis.SHAP import shap_analysis
from sklearn.ensemble import AdaBoostRegressor
from src.feature_engineering import average_daily
from objective_function import objective_adaboost
from src.Utils.Z_Score import remove_outliers_zscore
from src.model.train_model import get_X_y, train_model
from src.analysis.LIME import lime_sensitivity_analysis
from src.Optimiser.HOA.hoa_optimizer import hoa_optimizer

d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))


In [2]:
# Load raw data
DATA_PATH = "D:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\data\data.xlsx"
df = load_data(DATA_PATH)

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\hesam\AppData\Local\Temp\ipykernel_15300\2261336153.py:2: SyntaxWarning: invalid escape sequence '\M'
  DATA_PATH = "D:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\data\data.xlsx"


In [3]:
# Remove outliers using the function
cleaned_df, z_scores = remove_outliers_zscore(df, threshold=3)

# Display original and cleaned DataFrame
book = load_workbook(DATA_PATH)
if "Data after Z-score" in book.sheetnames:
    book.remove(book["Data after Z-score"])
    book.save(DATA_PATH)
with pd.ExcelWriter(DATA_PATH, engine="openpyxl", mode="a") as writer:
    cleaned_df.to_excel(writer, sheet_name="Data after Z-score", index=False)


X, y = get_X_y(cleaned_df, target_col="Burned Calories ")

Z-scores:
     Tracker Distance  Very Active Distance  Moderately Active Distance  \
0            0.651259              0.198319                   -0.099077   
1            0.088207              0.292342                    0.376262   
2           -1.373169             -0.565143                   -0.642321   
3            0.443954             -0.565143                   -0.257523   
4            0.610310             -0.437272                    0.183863   
..                ...                   ...                         ...   
935         -1.401322             -0.565143                   -0.642321   
936         -0.966236             -0.565143                   -0.246206   
937          0.479784             -0.365815                    1.655149   
938          0.487462             -0.328206                    1.247716   
939         -0.492760             -0.565143                   -0.642321   

     Light Active Distance  Very Active Minutes  Fairly Active Minutes  \
0              

In [4]:
# Apply K-Fold cross-validation
(
    X_train_best,
    X_test_best,
    y_train_best,
    y_test_best,
    K_Fold_Cross_Validation_Scores,
    combined_df,
) = K_Fold(X, y, n_splits=5)


# Save combined K-Fold data to Excel
book = load_workbook(DATA_PATH)
if "DATA after K-Fold" in book.sheetnames:
    book.remove(book["DATA after K-Fold"])
    book.save(DATA_PATH)
with pd.ExcelWriter(DATA_PATH, engine="openpyxl", mode="a") as writer:
    combined_df.to_excel(writer, sheet_name="DATA after K-Fold", index=False)


🔁 Fold 1 ------------------
  ADAboost → R2: 0.4823, RMSE: 202102.9122

🔁 Fold 2 ------------------
  ADAboost → R2: 0.4667, RMSE: 200856.6243

🔁 Fold 3 ------------------
  ADAboost → R2: 0.4678, RMSE: 194209.5324

🔁 Fold 4 ------------------
  ADAboost → R2: 0.4998, RMSE: 176575.3488

🔁 Fold 5 ------------------
  ADAboost → R2: 0.5366, RMSE: 180081.3100

✅ K-Fold Cross-Validation completed.
Best fold index: 4, R2: 0.5366
✅ Combined DataFrame using original target column name:


In [5]:
# Helper function to average metrics
def summarize_metrics(metrics_list):
    return {key: np.mean([m[key] for m in metrics_list]) for key in metrics_list[0]}


avg_metrics_k_fold = summarize_metrics(K_Fold_Cross_Validation_Scores)
print("Average Metrics:")
for key, value in avg_metrics_k_fold.items():
    print(f"{key}: {value:.4f}")

Average Metrics:
R2: 0.4907
RMSE: 190765.1456


In [6]:
singleModel_result = train_model(X_train_best, y_train_best, X_test_best, y_test_best)

d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))


In [7]:
best_pos, best_RMSE, convergence = hoa_optimizer(
    objective_adaboost,  # our AdaBoost objective
    [50, 0.01],  # lower bounds: n_estimators, learning_rate
    [300, 1.0],  # upper bounds
    2,  # dim
    10,  # n_agents
    10,  # max_iter
    X_train_best,
    y_train_best,
    X_test_best,
    y_test_best,
)



🥾 Iter 1/10 - Best RMSE: 353.08348
🥾 Iter 2/10 - Best RMSE: 353.08348
🥾 Iter 3/10 - Best RMSE: 351.82757
🥾 Iter 4/10 - Best RMSE: 351.82757
🥾 Iter 5/10 - Best RMSE: 351.82757
🥾 Iter 6/10 - Best RMSE: 350.92794
🥾 Iter 7/10 - Best RMSE: 350.92794
🥾 Iter 8/10 - Best RMSE: 350.92794
🥾 Iter 9/10 - Best RMSE: 350.92794
🥾 Iter 10/10 - Best RMSE: 350.92794


In [8]:
HOA_model_result = train_model(
    X_train_best, y_train_best, X_test_best, y_test_best, best_pos
)

d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))


In [9]:
# SHAP on the HOA model
sensitivity_df_shap, shap_values = shap_analysis(
    model=HOA_model_result["model"],  # assuming train_model returns dict with "model"
    X_train=X_train_best,
    y_train=y_train_best,
    X_test=X_test_best,
    y_test=y_test_best,
    save_path=DATA_PATH,  # save to same Excel file
    sheet_name="SHAP_Sensitivity",
)

  0%|          | 0/172 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
sensitivity_LIME = lime_sensitivity_analysis(
    model=HOA_model_result["model"],
    X_train=X_train_best,
    y_train=y_train_best,
    X_test=X_test_best,
    y_test=y_test_best,
    sample_index=5,
    epsilon=0.05,
)



📊 LIME Sensitivity for each feature:
Very Active Minutes > 29.00: 24.7143
5.03 < Tracker Distance <= 7.14: 11.9335
3.27 < Light Active Distance <= 4.78: 4.8021
Very Active Distance > 1.82: 2.8707
Sedentary Minutes <= 738.75: 8.2220
5.00 < Fairly Active Minutes <= 18.00: 13.5337
197.00 < Lightly Active Minutes <= 266.00: 6.0718
7133.50 < Steps <= 10132.75: 9.7169
Moderately Active Distance > 0.73: 11.6844


d:\Python\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(


In [ ]:





print("\nBest AdaBoost Params:", best_pos)
print("Best RMSE:", best_RMSE)